In [4]:
import pandas as pd
from utils import Graph

ModuleNotFoundError: No module named 'utils'

In [4]:
competitors = pd.read_parquet('data/competitors.parquet.gzip')
parents = pd.read_parquet('data/parents.parquet.gzip')
partners = pd.read_parquet('data/partners.parquet.gzip')
suppliers = pd.read_parquet('data/suppliers.parquet.gzip')


In [1]:
cypher_query = '''
MATCH (movie:Movie {title:"The Matrix"})<-[:ACTED_IN]-(actor)-[:ACTED_IN]->(rec:Movie)
 RETURN distinct rec.title as title LIMIT 20
'''

first = Graph("bolt://44.204.52.109:7687", "neo4j","card-twirls-workbook")
print(first.write_query(cypher_query))

NameError: name 'Graph' is not defined

In [6]:
competitors.head()

,source_id,target_id,start_date,source_name,targte_name,type
0,50472,58267,2020-03-23,"Infinity Pharmaceuticals, Inc.",Innate Pharma SA,COMPETES
1,282940,4383,2020-03-23,"Bozhon Precision Industry Technology Co., Ltd.",National Instruments Corp.,COMPETES
2,50472,247699,2020-03-23,"Infinity Pharmaceuticals, Inc.","NextCure, Inc.",COMPETES
3,313953,29664,2020-03-23,Quaker Houghton,"Yushiro Chemical Industry Co., Ltd.",COMPETES
4,313953,129287,2020-03-23,Quaker Houghton,Petrofer GmbH,COMPETES


In [7]:
parents.head()

,source_id,target_id,start_date,source_name,targte_name,type
0,265502,369980,2022-04-01,Linde Plc,"Wahlco Environmental Systems, Inc.",ULTIMATE_PARENT_OF
1,318,3840,2022-04-01,"Verizon Communications, Inc.",Western Wireless Corp.,ULTIMATE_PARENT_OF
2,292835,3841,2022-04-01,Truist Financial Corp.,"Main Street Banks, Inc. (Atlanta, Georgia)",ULTIMATE_PARENT_OF
3,1971,3843,2022-04-01,"The PNC Financial Services Group, Inc.",Yardville National Bancorp,ULTIMATE_PARENT_OF
4,25278,3844,2022-04-01,"Toray Industries, Inc.","Zoltek Cos., Inc.",ULTIMATE_PARENT_OF


In [8]:
partners.head()

,source_id,target_id,start_date,source_name,targte_name,type
0,323302,97940,2021-12-08,"& Factory, Inc.",Amix Co. Ltd.,PARTNERS
1,323302,313319,2021-12-08,"& Factory, Inc.","Mirror Fit Co., Ltd.",PARTNERS
2,323302,150832,2021-12-08,"& Factory, Inc.",Housecom Corp.,PARTNERS
3,323302,313325,2021-12-08,"& Factory, Inc.",Miyoshi Real Estate Co. Ltd.,PARTNERS
4,323302,137558,2021-12-08,"& Factory, Inc.",Maeda & Co. Ltd.,PARTNERS


In [9]:
suppliers.head()

,source_id,target_id,revenue_pct,start_date,source_name,targte_name,type
0,46216,317277,NaN,2018-08-25,Ever Team Software SAS,GFI Informatique SA,SUPPLIES
1,323302,362405,12.0,2021-12-08,"& Factory, Inc.","Shogakukan, Inc.",SUPPLIES
2,323302,96117,34.1,2021-12-08,"& Factory, Inc.","Square Enix Holdings Co., Ltd.",SUPPLIES
3,323302,362527,19.5,2021-12-08,"& Factory, Inc.","Shueisha, Inc.",SUPPLIES
4,323304,282371,NaN,2021-02-17,(B. Y.) Medimor Ltd.,InMode Ltd.,SUPPLIES
